# Module 11 Challenge
## Deliverable 1: Scrape Titles and Preview Text from Mars News

In [1]:
# Scraping
from splinter import Browser
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# requests
import requests
import time

# parsing
from bs4 import BeautifulSoup

# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Open a Chrome window using Splinter
my_service = Service(executable_path=ChromeDriverManager().install())
browser = Browser('chrome', service=my_service)

### Step 1: Visit the Website

1. Use automated browsing to visit the [Mars news site](https://static.bc-edx.com/data/web/mars_news/index.html). Inspect the page to identify which elements to scrape.

      > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools.

In [5]:
# Visit the Mars news site
url = 'https://static.bc-edx.com/data/web/mars_news/index.html'
browser.visit(url)

### Step 2: Scrape the Website

Create a Beautiful Soup object and use it to extract text elements from the website.

In [6]:
# Create a Beautiful Soup object
# Gather the html from the current page in the browser
html = browser.html

# soupify
soup = BeautifulSoup(html, 'html.parser')

In [7]:
# Extract all the text elements
# Find <div id="news" class="container "> (parent div for the title) and assign to parent_container
parent_container = soup.find("div", {"id": "news"})

# Find <div class="list_text"> under the parent and assign to articles 
articles = parent_container.find_all("div", {"class": "list_text"})

# Find <div class="list_text"> under the parent and assign to articles - per result there are 15 titles to scrape
print(len(articles))

15


### Step 3: Store the Results

Extract the titles and preview text of the news articles that you scraped. Store the scraping results in Python data structures as follows:

* Store each title-and-preview pair in a Python dictionary. And, give each dictionary two keys: `title` and `preview`. An example is the following:

  ```python
  {'title': "NASA's MAVEN Observes Martian Light Show Caused by Major Solar Storm", 
   'preview': "For the first time in its eight years orbiting Mars, NASA’s MAVEN mission witnessed two different types of ultraviolet aurorae simultaneously, the result of solar storms that began on Aug. 27."
  }
  ```

* Store all the dictionaries in a Python list.

* Print the list in your notebook.

In [10]:
# test for one article

article = articles[0]

# extract date, title and text under the article (assigned above).  Text strip removes spaces from the end.
# date under <div class="list_date">
# title under <div class="content_title">
# preview under <div class="article_teaser_body">
date = article.find("div", {"class": "list_date"}).text.strip()
title = article.find("div", {"class": "content_title"}).text.strip()
preview = article.find("div", {"class": "article_teaser_body"}).text.strip()

print(date, title, preview)

November 9, 2022 NASA's MAVEN Observes Martian Light Show Caused by Major Solar Storm For the first time in its eight years orbiting Mars, NASA’s MAVEN mission witnessed two different types of ultraviolet aurorae simultaneously, the result of solar storms that began on Aug. 27.


In [12]:
#Now that we know we have the correct information for the title and preview above (I intentionally included date as well), 
#We can now loop through the HTML to find all titles for the Data Frame.
#Loop through article (assigned above) to retrieve the variables needed.  Assign to date, title and preview.  Append in the loop as new rows are found.
#Quality check, make sure there are 15 as shown above from print(len(articles))
rows = []

for article in articles:
    # extract 
    date = article.find("div", {"class": "list_date"}).text.strip()
    title = article.find("div", {"class": "content_title"}).text.strip()
    preview = article.find("div", {"class": "article_teaser_body"}).text.strip()

    # create the row
    row = {
        "article_date": date,
        "article_title": title,
        "article_preview": preview
    }
    rows.append(row)

df = pd.DataFrame(rows)
df

,article_date,article_title,article_preview
0,"November 9, 2022",NASA's MAVEN Observes Martian Light Show Cause...,For the first time in its eight years orbiting...
1,"November 1, 2022",NASA Prepares to Say 'Farewell' to InSight Spa...,A closer look at what goes into wrapping up th...
2,"October 28, 2022",NASA and ESA Agree on Next Steps to Return Mar...,The agency’s Perseverance rover will establish...
3,"October 27, 2022",NASA's InSight Lander Detects Stunning Meteoro...,The agency’s lander felt the ground shake duri...
4,"October 21, 2022","NASA To Host Briefing on InSight, Mars Reconna...",Scientists from two Mars missions will discuss...
5,"October 20, 2022",Why NASA Is Trying To Crash Land on Mars,"Like a car’s crumple zone, the experimental SH..."
6,"October 19, 2022",Curiosity Mars Rover Reaches Long-Awaited Salt...,"After years of climbing, the Mars rover has ar..."
7,"October 12, 2022",Mars Mission Shields Up for Tests,Protecting Mars Sample Return spacecraft from ...
8,"October 7, 2022",NASA's InSight Waits Out Dust Storm,InSight’s team is taking steps to help the sol...
9,"September 19, 2022",NASA's InSight 'Hears' Its First Meteoroid Imp...,The Mars lander’s seismometer has picked up vi...


In [13]:
#View the data types to determine if any data cleaning needs to occur
#Notice below Date is an object.  Since we pulled date, it can be cleaned up.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   article_date     15 non-null     object
 1   article_title    15 non-null     object
 2   article_preview  15 non-null     object
dtypes: object(3)
memory usage: 488.0+ bytes


In [15]:
#convert date from object to datetime
df["article_date"] = pd.to_datetime(df["article_date"])
df.head()

,article_date,article_title,article_preview
0,2022-11-09,NASA's MAVEN Observes Martian Light Show Cause...,For the first time in its eight years orbiting...
1,2022-11-01,NASA Prepares to Say 'Farewell' to InSight Spa...,A closer look at what goes into wrapping up th...
2,2022-10-28,NASA and ESA Agree on Next Steps to Return Mar...,The agency’s Perseverance rover will establish...
3,2022-10-27,NASA's InSight Lander Detects Stunning Meteoro...,The agency’s lander felt the ground shake duri...
4,2022-10-21,"NASA To Host Briefing on InSight, Mars Reconna...",Scientists from two Mars missions will discuss...


In [16]:
#Verify Data was converted.  Above it appears that it was.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   article_date     15 non-null     datetime64[ns]
 1   article_title    15 non-null     object        
 2   article_preview  15 non-null     object        
dtypes: datetime64[ns](1), object(2)
memory usage: 488.0+ bytes


In [18]:
#close the Browser and Dev Tools
browser.quit()

In [22]:
df.to_csv('part_1_mars_news_results.csv', index=False)